In [ ]:
# DON'T USE THIS FILE HERE IS THE SHARE LINK:
"""
https://drive.google.com/file/d/1D-BAbPYIFShVT4kXYyN8Yns1PX4Nsgep/view?usp=sharing
"""

# Authors: Matthew Cutrona & Garrett Weaver
# TRAIN PKL MODEL

import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression model
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# Save the model
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model trained and saved as model.pkl")

Model trained and saved as model.pkl


In [10]:
# SERVER

from flask import Flask, request, jsonify
import pickle
import numpy as np
import logging
import threading

app = Flask(__name__)

# Configure logging
logging.basicConfig(filename='server.log', level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s')

# Load the pre-trained model
try:
    with open('model.pkl', 'rb') as file:
        model = pickle.load(file)
    app.logger.info('Model loaded successfully.')
except Exception as e:
    app.logger.error(f'Error loading model: {e}')
    model = None

@app.route('/')
def home():
    return 'Hello, World! This is the ML model API.'

@app.route('/predict', methods=['POST'])
def predict():
    if model is None:
        return jsonify({'error': 'Model not loaded.'}), 500
    try:
        data = request.get_json(force=True)
        if 'features' not in data:
            return jsonify({'error': "The 'features' key is missing from the request payload."}), 400
        features = data['features']
        if not isinstance(features, list):
            return jsonify({'error': "'features' should be a list of numerical values."}), 400
        if len(features) != 4:
            return jsonify({'error': 'Exactly 4 feature values are required.'}), 400
        try:
            features = [float(x) for x in features]
        except ValueError:
            return jsonify({'error': 'All feature values must be numeric.'}), 400
        prediction = model.predict([features])
        return jsonify({'prediction': int(prediction[0])})
    except Exception as e:
        app.logger.error(f'Exception during prediction: {e}')
        return jsonify({'error': 'An error occurred during prediction.'}), 500

def run_flask_app():
    app.run(debug=True, use_reloader=False)

if __name__ == '__main__':
    flask_thread = threading.Thread(target=run_flask_app)
    flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: on


In [12]:
# CLIENT
import requests
import pickle

# The URL for the predict route
url = 'http://127.0.0.1:5000/predict'

# Example input features
data = {
    'features': [5.1, 3.5, 1.4, 0.2]
}

# Send a POST request to the server
response = requests.post(url, json=data)

# Print the prediction result
print(response.json())

INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 23:11:17] "POST /predict HTTP/1.1" 200 -


{'prediction': 0}


In [11]:
# CURL CLIENT
!curl -X POST -H "Content-Type: application/json" -d '{"features": [5.1, 3.5, 1.4, 0.2]}' http://127.0.0.1:5000/predict

INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 23:11:11] "POST /predict HTTP/1.1" 200 -


{
  "prediction": 0
}
